# Tumor Classification

## Libraries

In [ ]:
from keras.models import load_model
from keras.models import Model
from keras import backend as K

import matplotlib.pyplot as plt
import pydicom as pdc
import pandas as pd
import numpy as np
import models
import json
import math
import sys
import os
import gc

%matplotlib inline

## Path to the MRI Scan of Whole Brain / First Plane of Brain

In [ ]:
path = 'source/PILO/Stanford/ST_PF-PILO_T2_Axial/no_roi/ST_PF-PILO_M_0046-01-IM-0786-0001.dcm'

## Function to read the Brain MRI

In [ ]:
def load_volume(img_path):
    min_max = json.load(open(os.path.join('meta', 'clf_meta.json'), 'r'))
    volume = None
    
    if img_path[-4:] == '.dcm':
        volume = []
        img_path = '-'.join(img_path.split('-')[:-1])
        img_path = img_path.split(os.sep)
        img_path, filename = '{}'.format(os.sep).join(img_path[:-1]), img_path[-1]
        for _, _, files in os.walk(img_path):
            files = sorted(filter(lambda x: x.startswith(filename), files))
            for file in files:
                data = pdc.dcmread(os.path.join(img_path, file))
                volume.append(data.pixel_array.tolist())
        volume = np.asarray(volume)
    else:
        volume = np.load(img_path)['T2 ax'].transpose((2,0,1))
    
    volume = (volume - volume.mean())/volume.std()
    volume = 255 * (volume - min_max['min']) / (min_max['max'] - min_max['min'])
    volume = np.clip(volume, 0, 255)
    volume = np.rot90(volume, axes=(2,1))
    return np.repeat(np.expand_dims(volume, axis=-1), 3, axis=-1)

## Function to fetch Classifier

In [ ]:
def build_classifier(key, dropout_rate):
    preprocess = None
    model = None
    model_d = {'densenet': 'dn121', 'inceptionresnet': 'irv2',
               'inception': 'iv3', 'resnet': 'r50',
               'vgg': 'vgg', 'xception': 'x'}
    if key == 'densenet':
        from models.densenet import get_classifier, get_preprocess
    elif key == 'inceptionresnet':
        from models.inceptionresnet import get_classifier, get_preprocess
    elif key == 'inception':
        from models.inception import get_classifier, get_preprocess
    elif key == 'resnet':
        from models.resnet import get_classifier, get_preprocess
    elif key == 'vgg':
        from models.vgg import get_classifier, get_preprocess
    else:
        from models.xception import get_classifier, get_preprocess

    model_path = os.path.join('weights', 'clf_{}_{}.h5'.format(key, dropout_rate))
    model = load_model(model_path)
    model = Model(inputs=model.layers[0].input,
                  outputs=model.layers[-2].output,
                  name=key)
    return (model, get_preprocess())

## Function to fetch Paraclassifier

In [ ]:
def build_paraclassifier(key):
    model = None
    model_d = {'densenet': 'dn121', 'inceptionresnet': 'irv2',
               'inception': 'iv3', 'resnet': 'r50',
               'vgg': 'vgg', 'xception': 'x', 'ensemble': 'ensemble'}
    if key == 'densenet':
        from models.densenet import get_paraclassifier
    elif key == 'inceptionresnet':
        from models.inceptionresnet import get_paraclassifier
    elif key == 'inception':
        from models.inception import get_paraclassifier
    elif key == 'resnet':
        from models.resnet import get_paraclassifier
    elif key == 'vgg':
        from models.vgg import get_paraclassifier
    else:
        from models.xception import get_paraclassifier

    model_path = os.path.join('weights', 'para_{}.h5'.format(key))
    model = load_model(model_path)
    return model

## Inference

### Result from Best Individual Model - DenseNet

In [ ]:
mapping = {0: 'DIPG', 1: 'EP', 2: 'MB', 3: 'PILO', 4: 'Normal'}

volume = load_volume(path)
config = json.load(open(os.path.join('logs', 'clf_config.json'), 'r'))

model, preprocess = build_classifier('densenet', config['densenet'])
vector = model.predict(preprocess(volume))
del model
del preprocess
_ = gc.collect()

vector = np.expand_dims(vector, axis=0)
model = build_paraclassifier('ensemble')
prediction = model.predict(vector)

del model
_ = gc.collect()

for x in prediction[0].argsort()[-3:][::-1]:
    print('{} - {:.2f}%'.format(mapping[x], prediction[0, x] * 100))

### Result from Ensemble Model

In [ ]:
mapping = {0: 'DIPG', 1: 'EP', 2: 'MB', 3: 'PILO', 4: 'Normal'}

volume = load_volume(path)
config = json.load(open(os.path.join('logs', 'clf_config.json'), 'r'))
vectors = None

for key in sorted(config.keys()):
    print('Fetching vectors from {}'.format(key))
    K.clear_session()
    model, preprocess = build_classifier(key, config[key])
    if vectors is None:
        vectors = model.predict(preprocess(volume))
    else:
        vectors += model.predict(preprocess(volume))
    del model
    del preprocess
    _ = gc.collect()

    vectors /= len(config.keys())
vectors = np.expand_dims(vectors, axis=0)

model = build_paraclassifier('ensemble')
prediction = model.predict(vectors)

del model
_ = gc.collect()

print()
for x in prediction[0].argsort()[-3:][::-1]:
    print('{} - {:.2f}%'.format(mapping[x], prediction[0, x] * 100))